# FastHTML tests

In [ ]:
import time
from fastcore.utils import *

from collections import namedtuple
from IPython import display
from enum import Enum
from pprint import pprint

from fastcore.test import *
from starlette.testclient import TestClient
from starlette.responses import Response
from datetime import datetime
from dataclasses import dataclass, asdict

from fasthtml.all import *

In [ ]:
snake2hyphens("snake_case")

'Snake-Case'

In [ ]:
get_key()

'08b63b51-be3a-4f54-8d26-4cd27eb17c0d'

In [ ]:
app = FastHTML(secret_key='soopersecret')
cli = TestClient(app)
rt = app.route

In [ ]:
@rt("/hi")
def get(): return 'Hi there'

r = cli.get('/hi')
r.text

'Hi there'

In [ ]:
@rt("/hi")
def post(): return 'Postal'

cli.post('/hi').text

'Postal'

In [ ]:
@app.get("/")
def show_host(req): return req.headers['host']

cli.get('/').text

'testserver'

In [ ]:
@rt('/user/{nm}', name='gday')
def get(nm:str): return f"Good day to you, {nm}!"

cli.get('/user/Alexis').text

'Good day to you, Alexis!'

In [ ]:
test_eq(app.router.url_path_for('gday', nm='Jeremy'), '/user/Jeremy')

In [ ]:
def test_r(cli, path, exp, meth='get', **kwargs): test_eq(getattr(cli, meth)(path, **kwargs).text, exp)

app.chk = 'foo'
ModelName = str_enum('ModelName', "alexnet", "resnet", "lenet")
fake_db = [{"name": "Foo"}, {"name": "Bar"}]

In [ ]:
@rt('/html/{idx}')
async def get(idx:int): return Body(H4(f'Next is {idx+1}.'))

reg_re_param("imgext", "ico|gif|jpg|jpeg|webm")

@rt(r'/static/{path:path}{fn}.{ext:imgext}')
def get(fn:str, path:str, ext:str): return f"Getting {fn}.{ext} from /{path}"

@rt("/models/{nm}")
def get(nm:ModelName): return nm

@rt("/files/{path}")
async def get(path: Path): return path.with_suffix('.txt')

@rt("/items/")
def get(idx:int|None = 0): return fake_db[idx]

In [ ]:
test_r(cli, '/html/1', '<body>\n  <h4>\nNext is 2.\n  </h4>\n</body>\n')
test_r(cli, '/static/foo/jph.ico', 'Getting jph.ico from /foo/')
test_r(cli, '/models/alexnet', 'alexnet')
test_r(cli, '/files/foo', 'foo.txt')
test_r(cli, '/items/?idx=1', '{"name":"Bar"}')
test_r(cli, '/items/', '{"name":"Foo"}')

In [ ]:
@app.get("/booly/")
def _(coming:bool=True): return 'Coming' if coming else 'Not coming'

@app.get("/datie/")
def _(d:date): return d

@app.get("/ua")
async def _(user_agent:str): return user_agent

@app.get("/hxtest")
def _(htmx): return htmx.request

@app.get("/hxtest2")
def _(foo:HtmxHeaders, req): return foo.request

@app.get("/app")
def _(app): return app.chk

@app.get("/app2")
def _(foo:FastHTML): return foo.chk,HttpHeader("mykey", "myval")

In [ ]:
test_r(cli, '/booly/?coming=true', 'Coming')
test_r(cli, '/booly/?coming=no', 'Not coming')
date_str = "17th of May, 2024, 2p"
test_r(cli, f'/datie/?d={date_str}', '2024-05-17 14:00:00')
test_r(cli, '/ua', 'FastHTML', headers={'User-Agent':'FastHTML'})
test_r(cli, '/hxtest' , '1', headers={'HX-Request':'1'})
test_r(cli, '/hxtest2', '1', headers={'HX-Request':'1'})
test_r(cli, '/app' , 'foo')

In [ ]:
r = cli.get('/app2')
test_eq(r.text, 'foo\n')
test_eq(r.headers['mykey'], 'myval')

In [ ]:
@dataclass
class Bodie: a:int;b:str

@rt("/bodie/{nm}/")
async def post(nm:str, data:Bodie):
    res = asdict(data)
    res['nm'] = nm
    return res

@app.post("/bodied/")
async def bodied(nm:str, data:dict): return data

nt = namedtuple('Bodient', ['a','b'])

@app.post("/bodient/")
async def bodient(nm:str, data:nt): return data._asdict()

class Bodie2:
    a:int|None; b:str
    def __init__(self, a, b='foo'): store_attr()

@app.post("/bodie2/")
async def bodie(d:Bodie2): return f"a: {d.a}; b: {d.b}"

In [ ]:
d = dict(a=1, b='foo')

test_r(cli, '/bodie/me', '{"a":1,"b":"foo","nm":"me"}', 'post', data=d)
test_r(cli, '/bodied/', '{"a":"1","b":"foo"}', 'post', data=d)
test_r(cli, '/bodie2/', 'a: 1; b: foo', 'post', data={'a':1})
test_r(cli, '/bodient/', '{"a":"1","b":"foo"}', 'post', data=d)

In [ ]:
@rt("/setcookie")
async def get(req):
    now = datetime.now()
    res = Response(f'Set to {now}')
    res.set_cookie('now', str(now))
    return res

@rt("/getcookie")
async def get(now:date): return f'Cookie was set at time {now.time()}'

In [ ]:
print(cli.get('/setcookie').text)
time.sleep(0.01)
cli.get('/getcookie').text

Set to 2024-06-01 19:48:25.907960


'Cookie was set at time 19:48:25.907960'

In [ ]:
@rt("/setsess")
async def get(sess):
    now = datetime.now()
    sess['noo'] = str(now)
    return f'Set to {now}'

@rt("/getsess")
async def get(noo:date): return f'Session time: {noo.time()}'

In [ ]:
print(cli.get('/setsess').text)
time.sleep(0.01)
cli.get('/getsess').text

Set to 2024-06-01 19:48:27.190641


'Session time: 19:48:27.190641'

## fin -